# Day 4-5교시: 가변 인수 + 언패킹 통합

**학습 시간**: 50분  
**학습 목표**:

- `*args`로 개수가 정해지지 않은 인수 받기
- `**kwargs`로 키워드 인수들을 딕셔너리로 받기
- 리스트/딕셔너리 언패킹으로 데이터 펼치기
- FastAPI에서의 실전 활용 패턴 이해하기

---

## 📖 가변 인수와 언패킹이란?

### 왜 가변 인수가 필요할까요?

우리가 지금까지 만든 함수들은 매개변수 개수가 고정되어 있었습니다:

```python
def add(a, b):
    return a + b
```

이 함수는 딱 2개의 숫자만 더할 수 있습니다. 그런데 만약 **개수가 정해지지 않은** 숫자들을 모두 더하고 싶다면? 3개, 5개, 10개... 몇 개가 들어올지 모르는 상황이라면?

```python
# 이렇게 하고 싶은데...
add(1, 2)
add(1, 2, 3)
add(1, 2, 3, 4, 5)
```

**바로 이럴 때 `*args`를 사용합니다!** ✨

### 언패킹(Unpacking)은 뭔가요?

언패킹은 **"포장을 풀다"**라는 뜻입니다. 리스트나 딕셔너리 같은 컬렉션을 개별 요소들로 펼치는 것을 말합니다.

```python
# 리스트를 개별 인수로 펼치기
numbers = [1, 2, 3]
add(*numbers)  # add(1, 2, 3)과 동일!
```

### FastAPI에서는 어떻게 쓰일까요?

FastAPI로 API를 만들 때, 이런 패턴들이 **엄청 많이** 사용됩니다:

1. **설정 병합**: 기본 설정과 사용자 설정을 합칠 때
   ```python
   settings = {**default_config, **user_config}
   ```

2. **유연한 데이터 생성**: 필수 필드 + 선택 필드를 자유롭게
   ```python
   user = {"id": 1, "name": "Alice", **extra_fields}
   ```

3. **로깅 시스템**: 다양한 정보를 유연하게 기록
   ```python
   log("ERROR", "Database failed", code=500, retry=True)
   ```

> 💡 **핵심**: `*args`와 `**kwargs`는 함수를 "유연하게" 만들어줍니다. 정해진 개수의 인수만 받는 게 아니라, 상황에 따라 다양하게 받을 수 있죠!

---

## 1. `*args` - 여러 개의 인수 받기

### `*args`는 "가변 위치 인수"입니다

`*args`는 **개수가 정해지지 않은 위치 인수**들을 **튜플**로 받습니다.

- `*`(별표): "여러 개를 받겠다"는 표시
- `args`: 변수명 (arguments의 약자, 다른 이름도 가능하지만 관례적으로 args 사용)
- 함수 내부에서 `args`는 **튜플** 형태로 사용됩니다

### 어떻게 동작하나요?

함수를 호출할 때 전달한 **모든 위치 인수**들이 하나의 튜플로 묶여서 전달됩니다.

In [ ]:
# *args는 여러 개의 인수를 튜플로 받습니다
def add_all(*args):
    """모든 숫자를 더하는 함수"""
    print("받은 인수들:", args)  # args는 튜플입니다!
    print("타입 확인:", type(args))  # <class 'tuple'>
    
    total = 0
    for num in args:  # 튜플을 순회하며 합계 계산
        total += num
    return total

# 인수 개수에 상관없이 호출 가능!
print("\n=== 2개 전달 ===")
result1 = add_all(1, 2, 3)
print("합계:", result1)  # 출력: 6

print("\n=== 5개 전달 ===")
result2 = add_all(10, 20, 30, 40, 50)
print("합계:", result2)  # 출력: 150

print("\n=== 1개만 전달 ===")
result3 = add_all(5)
print("합계:", result3)  # 출력: 5

**실행 결과:**
```
=== 2개 전달 ===
받은 인수들: (1, 2, 3)
타입 확인: <class 'tuple'>
합계: 6

=== 5개 전달 ===
받은 인수들: (10, 20, 30, 40, 50)
타입 확인: <class 'tuple'>
합계: 150

=== 1개만 전달 ===
받은 인수들: (5,)
타입 확인: <class 'tuple'>
합계: 5
```

### 💡 핵심 포인트

1. `*args`는 함수 내부에서 **튜플**로 사용됩니다
2. 전달한 모든 위치 인수가 순서대로 튜플에 담깁니다
3. 개수 제한이 없어서 0개, 1개, 100개 모두 가능합니다
4. `for`문으로 순회하거나 인덱싱(`args[0]`)으로 접근 가능합니다

### ⚠️ 주의사항

- `*args`는 **위치 인수**만 받습니다 (키워드 인수는 안 됨)
- 변수명이 `args`일 필요는 없지만, **관례적으로 `args`를 사용**합니다
- 일반 매개변수와 함께 사용할 때는 `*args`를 **나중**에 써야 합니다

---

## 2. `**kwargs` - 키워드 인수들 받기

### `**kwargs`는 "가변 키워드 인수"입니다

`**kwargs`는 **개수가 정해지지 않은 키워드 인수**들을 **딕셔너리**로 받습니다.

- `**`(별표 2개): "여러 개의 키워드 인수를 받겠다"는 표시
- `kwargs`: 변수명 (keyword arguments의 약자, 관례적으로 kwargs 사용)
- 함수 내부에서 `kwargs`는 **딕셔너리** 형태로 사용됩니다

### `*args`와의 차이점

| 구분 | `*args` | `**kwargs` |
|------|---------|------------|
| 받는 인수 | 위치 인수 | 키워드 인수 |
| 데이터 타입 | 튜플 | 딕셔너리 |
| 호출 예시 | `func(1, 2, 3)` | `func(a=1, b=2, c=3)` |

In [ ]:
# **kwargs는 키워드 인수들을 딕셔너리로 받습니다
def print_user_info(**kwargs):
    """사용자 정보를 출력하는 함수"""
    print("받은 정보:", kwargs)  # kwargs는 딕셔너리입니다!
    print("타입 확인:", type(kwargs))  # <class 'dict'>
    
    print("\n상세 정보:")
    for key, value in kwargs.items():  # 딕셔너리 순회
        print(f"  {key}: {value}")

# 키워드 인수를 자유롭게 전달
print("=== 사용자 1 ===")
print_user_info(name="홍길동", age=25, city="서울")

print("\n=== 사용자 2 ===")
print_user_info(email="test@example.com", phone="010-1234-5678")

print("\n=== 많은 정보 ===")
print_user_info(
    name="이영희",
    age=30,
    city="부산",
    job="개발자",
    hobby="독서"
)

**실행 결과:**
```
=== 사용자 1 ===
받은 정보: {'name': '홍길동', 'age': 25, 'city': '서울'}
타입 확인: <class 'dict'>

상세 정보:
  name: 홍길동
  age: 25
  city: 서울

=== 사용자 2 ===
받은 정보: {'email': 'test@example.com', 'phone': '010-1234-5678'}
타입 확인: <class 'dict'>

상세 정보:
  email: test@example.com
  phone: 010-1234-5678
```

### 💡 핵심 포인트

1. `**kwargs`는 함수 내부에서 **딕셔너리**로 사용됩니다
2. `키=값` 형태로 전달한 모든 인수가 딕셔너리에 담깁니다
3. `.items()`, `.keys()`, `.values()` 같은 딕셔너리 메서드 사용 가능
4. **어떤 키가 들어올지 모를 때** 유용합니다!

### 🎯 실무 활용 예시

FastAPI에서 로그를 남길 때 이런 식으로 사용합니다:

```python
# 상황마다 다른 정보를 기록
log("로그인 성공", user_id=123, ip="192.168.1.1")
log("주문 완료", order_id=456, amount=50000, payment="card")
log("에러 발생", error_code=500, message="DB 연결 실패")
```

---

## 3. 리스트 언패킹 (`*list`)

### 언패킹(Unpacking)이란?

리스트나 튜플 같은 **컬렉션을 개별 요소로 풀어헤치는** 것을 언패킹이라고 합니다.

```python
# 언패킹 전
numbers = [1, 2, 3]

# 언패킹 후 (*를 붙임)
*numbers  # → 1, 2, 3 (개별 요소로 펼쳐짐)
```

### 왜 필요한가요?

리스트에 데이터가 담겨있는데, 함수는 개별 인수로 받아야 할 때 사용합니다!

```python
# 상황: add_all 함수는 개별 숫자를 받음
add_all(1, 2, 3)  # 이건 되지만...

numbers = [1, 2, 3]
add_all(numbers)  # 이건 안 됨! (리스트 1개를 전달)
add_all(*numbers)  # 이렇게 언패킹! (1, 2, 3으로 펼쳐짐)
```

In [ ]:
# 리스트를 개별 인수로 펼치기
numbers = [10, 20, 30]

# 언패킹 없이 (에러 발생!)
try:
    result = add_all(numbers)  # 리스트 1개를 전달 → 에러!
except TypeError as e:
    print("에러 발생:", e)
    print("→ 리스트 전체가 하나의 인수로 전달되어서 더할 수 없음!\n")

# 언패킹 사용! (*를 붙임)
result = add_all(*numbers)  # add_all(10, 20, 30)과 동일!
print("언패킹 결과:", result)  # 출력: 60

print("\n=== 리스트 합치기 ===")
# 여러 리스트를 언패킹으로 합치기
list1 = [1, 2, 3]
list2 = [4, 5, 6]
list3 = [7, 8, 9]

# 언패킹으로 합치기
combined = [*list1, *list2, *list3]
print("합친 리스트:", combined)  # 출력: [1, 2, 3, 4, 5, 6, 7, 8, 9]

# 중간에 다른 값도 넣을 수 있음!
combined2 = [0, *list1, 100, *list2, 200]
print("응용:", combined2)  # 출력: [0, 1, 2, 3, 100, 4, 5, 6, 200]

**실행 결과:**
```
에러 발생: unsupported operand type(s) for +=: 'int' and 'list'
→ 리스트 전체가 하나의 인수로 전달되어서 더할 수 없음!

언패킹 결과: 60

=== 리스트 합치기 ===
합친 리스트: [1, 2, 3, 4, 5, 6, 7, 8, 9]
응용: [0, 1, 2, 3, 100, 4, 5, 6, 200]
```

### 💡 핵심 포인트

1. `*list`는 리스트의 요소들을 **개별적으로** 펼칩니다
2. 함수 호출 시: `func(*list)` → `func(요소1, 요소2, 요소3, ...)`
3. 리스트 생성 시: `[*list1, *list2]` → 여러 리스트 합치기
4. 튜플도 동일하게 언패킹 가능합니다

---

## 4. 딕셔너리 언패킹 (`**dict`)

### 딕셔너리 언패킹은 `**`를 사용합니다

리스트 언패킹이 `*` 하나였다면, 딕셔너리 언패킹은 `**` 두 개를 사용합니다.

```python
# 딕셔너리 언패킹
user = {"name": "홍길동", "age": 25}
**user  # → name="홍길동", age=25 (키워드 인수로 펼쳐짐)
```

### 언제 사용하나요?

1. **함수 호출**: 딕셔너리를 키워드 인수로 전달
2. **딕셔너리 병합**: 여러 딕셔너리를 합칠 때 (FastAPI 핵심!)

In [ ]:
# 딕셔너리를 키워드 인수로 펼치기
user_data = {"name": "이영희", "age": 30, "city": "부산"}

print("=== 함수 호출에 언패킹 사용 ===")
# 이렇게 하면 print_user_info(name="이영희", age=30, city="부산")과 동일!
print_user_info(**user_data)

print("\n=== 딕셔너리 병합 (FastAPI 핵심!) ===")
# 기본 설정
default_config = {"timeout": 30, "retry": 3, "debug": False}
print("기본 설정:", default_config)

# 사용자 설정 (일부만 변경)
user_config = {"timeout": 60, "debug": True}
print("사용자 설정:", user_config)

# 딕셔너리 언패킹으로 병합!
merged_config = {**default_config, **user_config}
print("\n병합된 설정:", merged_config)
# 출력: {'timeout': 60, 'retry': 3, 'debug': True}
# → timeout은 user_config 값(60)으로 덮어씌워짐!

print("\n=== 새 키 추가하면서 병합 ===")
final_config = {**merged_config, "log_level": "INFO", "port": 8000}
print("최종 설정:", final_config)

**실행 결과:**
```
=== 함수 호출에 언패킹 사용 ===
받은 정보: {'name': '이영희', 'age': 30, 'city': '부산'}
타입 확인: <class 'dict'>

상세 정보:
  name: 이영희
  age: 30
  city: 부산

=== 딕셔너리 병합 (FastAPI 핵심!) ===
기본 설정: {'timeout': 30, 'retry': 3, 'debug': False}
사용자 설정: {'timeout': 60, 'debug': True}

병합된 설정: {'timeout': 60, 'retry': 3, 'debug': True}

=== 새 키 추가하면서 병합 ===
최종 설정: {'timeout': 60, 'retry': 3, 'debug': True, 'log_level': 'INFO', 'port': 8000}
```

### 💡 핵심 포인트

1. `**dict`는 딕셔너리를 **키워드 인수**로 펼칩니다
2. 함수 호출: `func(**dict)` → `func(key1=value1, key2=value2, ...)`
3. **딕셔너리 병합**: `{**dict1, **dict2}` → 두 딕셔너리 합치기
4. 같은 키가 있으면 **나중 값이 우선**됩니다 (덮어쓰기)

### 🎯 FastAPI에서 정말 많이 씁니다!

```python
# 기본 응답 + 추가 정보
response = {
    "status": "success",
    **user_data,  # 사용자 정보 병합
    "timestamp": "2026-02-03"
}
```

---

## 5. `*args`와 `**kwargs` 조합

### 둘을 함께 사용할 수 있습니다!

하나의 함수에서 `*args`와 `**kwargs`를 **동시에** 사용하면 **최고로 유연한 함수**를 만들 수 있습니다.

**순서 규칙:**
1. 일반 매개변수
2. `*args` (위치 인수들)
3. `**kwargs` (키워드 인수들)

```python
def func(a, b, *args, **kwargs):
    pass
```

In [ ]:
def flexible_function(*args, **kwargs):
    """위치 인수와 키워드 인수를 모두 받는 유연한 함수"""
    print("=== 받은 인수들 ===")
    print("위치 인수 (*args):", args, f"(타입: {type(args).__name__})")
    print("키워드 인수 (**kwargs):", kwargs, f"(타입: {type(kwargs).__name__})")
    
    # 활용 예시
    if args:
        print(f"\n위치 인수 개수: {len(args)}개")
        print("첫 번째 위치 인수:", args[0])
    
    if kwargs:
        print(f"\n키워드 인수 개수: {len(kwargs)}개")
        for key, value in kwargs.items():
            print(f"  {key} = {value}")

# 다양한 방식으로 호출 가능!
print("\n🔹 케이스 1: 위치 인수만")
flexible_function(1, 2, 3)

print("\n🔹 케이스 2: 키워드 인수만")
flexible_function(name="홍길동", age=25)

print("\n🔹 케이스 3: 둘 다 사용")
flexible_function(1, 2, 3, name="홍길동", age=25)

print("\n🔹 케이스 4: 아무것도 안 줘도 됨")
flexible_function()

**실행 결과:**
```
🔹 케이스 1: 위치 인수만
=== 받은 인수들 ===
위치 인수 (*args): (1, 2, 3) (타입: tuple)
키워드 인수 (**kwargs): {} (타입: dict)

위치 인수 개수: 3개
첫 번째 위치 인수: 1

🔹 케이스 2: 키워드 인수만
=== 받은 인수들 ===
위치 인수 (*args): () (타입: tuple)
키워드 인수 (**kwargs): {'name': '홍길동', 'age': 25} (타입: dict)

키워드 인수 개수: 2개
  name = 홍길동
  age = 25

🔹 케이스 3: 둘 다 사용
=== 받은 인수들 ===
위치 인수 (*args): (1, 2, 3) (타입: tuple)
키워드 인수 (**kwargs): {'name': '홍길동', 'age': 25} (타입: dict)

위치 인수 개수: 3개
첫 번째 위치 인수: 1

키워드 인수 개수: 2개
  name = 홍길동
  age = 25
```

### 💡 핵심 포인트

1. `*args`는 튜플, `**kwargs`는 딕셔너리
2. 둘 다 비어있을 수 있습니다 (빈 튜플, 빈 딕셔너리)
3. 순서가 중요: 일반 → `*args` → `**kwargs`
4. **최대 유연성**: 어떤 형태의 인수든 받을 수 있음!

---

## 6. 실전 예제: 로그 함수

### 실무에서 이렇게 사용합니다!

지금까지 배운 내용을 종합해서 **프로덕션 레벨의 로그 함수**를 만들어봅시다.

**요구사항:**
- 로그 레벨 (INFO, ERROR 등) 필수
- 여러 개의 메시지를 받을 수 있어야 함
- 추가 정보(에러 코드, 재시도 여부 등)를 자유롭게 기록

이걸 `*args`와 `**kwargs`로 해결합니다!

In [ ]:
def log_message(level, *messages, **details):
    """로그 메시지를 출력하는 함수 (FastAPI 로깅 패턴)
    
    Args:
        level: 로그 레벨 (필수)
        *messages: 여러 개의 메시지
        **details: 추가 상세 정보
    """
    # 레벨 표시 (색상 표현을 위한 이모지)
    level_emoji = {
        "INFO": "ℹ️",
        "WARNING": "⚠️",
        "ERROR": "❌",
        "SUCCESS": "✅"
    }
    emoji = level_emoji.get(level, "📝")
    
    # 레벨과 메시지 출력
    print(f"{emoji} [{level}]", end=" ")
    
    # 모든 메시지를 공백으로 구분해서 출력
    for msg in messages:
        print(msg, end=" ")
    print()  # 줄바꿈
    
    # 상세 정보가 있으면 출력
    if details:
        print("  📋 세부사항:", details)

# 사용 예시
print("=== 다양한 로그 시나리오 ===\n")

# 1. 간단한 정보 로그
log_message("INFO", "사용자 로그인 성공")

# 2. 여러 메시지 조합
log_message("WARNING", "데이터베이스", "응답 시간", "지연")

# 3. 에러 로그 + 상세 정보
log_message(
    "ERROR",
    "데이터베이스", "연결 실패",
    error_code=500,
    retry=True,
    host="localhost"
)

# 4. 성공 로그
log_message(
    "SUCCESS",
    "파일 업로드 완료",
    filename="report.pdf",
    size="2.5MB",
    user_id=123
)

**실행 결과:**
```
=== 다양한 로그 시나리오 ===

ℹ️ [INFO] 사용자 로그인 성공 
⚠️ [WARNING] 데이터베이스 응답 시간 지연 
❌ [ERROR] 데이터베이스 연결 실패 
  📋 세부사항: {'error_code': 500, 'retry': True, 'host': 'localhost'}
✅ [SUCCESS] 파일 업로드 완료 
  📋 세부사항: {'filename': 'report.pdf', 'size': '2.5MB', 'user_id': 123}
```

### 💡 이 함수의 장점

1. **유연성**: 메시지 개수와 상세 정보를 자유롭게 조절
2. **확장성**: 나중에 다른 필드 추가해도 함수 수정 불필요
3. **가독성**: 호출할 때 무슨 정보를 기록하는지 명확

---

## 7. FastAPI 스타일 예제

### FastAPI에서 정말 많이 쓰이는 패턴!

FastAPI로 API를 만들 때, **필수 필드**와 **선택 필드**를 함께 받아야 하는 경우가 많습니다.

예를 들어:
- 사용자 생성: username, email은 필수, 나머지는 선택
- 상품 등록: name, price는 필수, description, image 등은 선택

이럴 때 `**extra_fields` 패턴을 사용합니다!

In [ ]:
def create_user(username, email, **extra_fields):
    """사용자 생성 함수 (FastAPI Pydantic 패턴 미리보기)
    
    Args:
        username: 사용자명 (필수)
        email: 이메일 (필수)
        **extra_fields: 추가 필드들 (선택)
    
    Returns:
        사용자 정보 딕셔너리
    """
    # 기본 사용자 정보 + 추가 필드 병합
    user = {
        "username": username,
        "email": email,
        **extra_fields  # 딕셔너리 언패킹으로 병합!
    }
    
    return user

# 사용 예시
print("=== 사용자 생성 시나리오 ===\n")

# 1. 필수 필드만
user1 = create_user("hong", "hong@example.com")
print("사용자 1 (필수만):", user1)
print()

# 2. 선택 필드 추가 (나이, 도시)
user2 = create_user(
    "kim",
    "kim@example.com",
    age=25,
    city="서울"
)
print("사용자 2 (선택 추가):", user2)
print()

# 3. 다양한 선택 필드
user3 = create_user(
    "lee",
    "lee@example.com",
    age=30,
    phone="010-1234-5678",
    address="부산시",
    job="개발자"
)
print("사용자 3 (많은 필드):", user3)

print("\n=== 언패킹 응용: 기존 데이터 활용 ===")
# 이미 있는 딕셔너리를 언패킹해서 전달
extra_info = {
    "age": 28,
    "city": "대구",
    "hobby": "독서"
}

user4 = create_user("park", "park@example.com", **extra_info)
print("사용자 4 (딕셔너리 언패킹):", user4)

**실행 결과:**
```
=== 사용자 생성 시나리오 ===

사용자 1 (필수만): {'username': 'hong', 'email': 'hong@example.com'}

사용자 2 (선택 추가): {'username': 'kim', 'email': 'kim@example.com', 'age': 25, 'city': '서울'}

사용자 3 (많은 필드): {'username': 'lee', 'email': 'lee@example.com', 'age': 30, 'phone': '010-1234-5678', 'address': '부산시', 'job': '개발자'}

=== 언패킹 응용: 기존 데이터 활용 ===
사용자 4 (딕셔너리 언패킹): {'username': 'park', 'email': 'park@example.com', 'age': 28, 'city': '대구', 'hobby': '독서'}
```

### 🎯 FastAPI에서 이렇게 발전합니다

우리가 배운 이 패턴이 FastAPI에서는 이렇게 사용됩니다:

```python
# FastAPI + Pydantic (나중에 배울 내용)
from pydantic import BaseModel

class UserBase(BaseModel):  # 기본 필드
    username: str
    email: str

class UserCreate(UserBase):  # 생성 시 추가 필드
    password: str
    age: int = None  # 선택
    city: str = None  # 선택

@app.post("/users")
def create_user(user: UserCreate):
    # user 객체가 자동으로 검증됨!
    return {"username": user.username, **user.dict()}
```

### 💡 핵심 포인트

1. **필수 + 선택** 패턴은 실무에서 정말 많이 사용됩니다
2. `**extra_fields`로 유연하게 확장 가능
3. 딕셔너리 언패킹(`**dict`)으로 깔끔하게 병합
4. 이 패턴이 Pydantic의 기초가 됩니다!

---

## 📌 핵심 정리

이번 시간에 배운 내용:

### 1. `*args` (가변 위치 인수)
- 개수가 정해지지 않은 위치 인수를 **튜플**로 받음
- 사용: `def func(*args):`
- 호출: `func(1, 2, 3, 4, 5)`

### 2. `**kwargs` (가변 키워드 인수)
- 개수가 정해지지 않은 키워드 인수를 **딕셔너리**로 받음
- 사용: `def func(**kwargs):`
- 호출: `func(name="홍길동", age=25, city="서울")`

### 3. 리스트 언패킹 (`*list`)
- 리스트를 개별 요소로 펼침
- 함수 호출: `func(*[1, 2, 3])` → `func(1, 2, 3)`
- 리스트 합치기: `[*list1, *list2]`

### 4. 딕셔너리 언패킹 (`**dict`)
- 딕셔너리를 키워드 인수로 펼침
- 함수 호출: `func(**{"a": 1, "b": 2})` → `func(a=1, b=2)`
- **딕셔너리 병합**: `{**dict1, **dict2}` ← FastAPI 핵심!

### 5. 조합 사용
- `def func(*args, **kwargs):` → 최대 유연성
- 순서: 일반 매개변수 → `*args` → `**kwargs`

---

## 🚀 다음 단계

**이제 실습할 차례입니다!**

1. **`2_practice.py`**: 강사와 함께 실습하며 익히기
   - `*args`로 합계 함수 만들기
   - `**kwargs`로 로그 함수 만들기
   - 딕셔너리 언패킹으로 설정 병합하기

2. **`3_exercise.py`**: 스스로 문제 풀어보기
   - 유연한 계산기 함수
   - 설정 병합 시스템
   - API 요청 함수 (도전!)

---

## 💡 기억할 점

### ⭐ 가장 중요한 3가지

1. **`*args`는 튜플, `**kwargs`는 딕셔너리**
   - 이것만 기억하면 나머지는 자연스럽게 이해됩니다!

2. **딕셔너리 언패킹 `{**dict1, **dict2}`**
   - FastAPI에서 정말 많이 사용됩니다
   - 같은 키가 있으면 나중 값이 우선!

3. **필수 + 선택 필드 패턴**
   ```python
   def create_user(username, email, **extra):
       return {"username": username, "email": email, **extra}
   ```
   - 이 패턴이 Pydantic BaseModel의 기초가 됩니다!

### ⚠️ 주의사항

- **순서 지키기**: 일반 매개변수 → `*args` → `**kwargs`
- **변수명**: `args`, `kwargs`는 관례입니다 (다른 이름도 가능하지만 비추천)
- **과용 금지**: 너무 유연하면 오히려 혼란스러울 수 있습니다

### 🎯 실무 팁

1. 함수가 받을 인수가 **명확하면** → 일반 매개변수 사용
2. 인수 개수가 **가변적**이면 → `*args` 사용
3. 옵션이 **많고 자주 바뀌면** → `**kwargs` 사용
4. **설정 병합**할 때 → 딕셔너리 언패킹 `{**dict1, **dict2}`

---

**여러분은 이제 Python의 가장 강력한 함수 패턴을 배웠습니다!** 🎉

이 패턴들을 잘 활용하면 FastAPI에서 정말 깔끔하고 유연한 코드를 작성할 수 있습니다. 다음 교시에서는 **타입 힌트**를 배워서 이 함수들을 더욱 안전하게 만들어볼 거예요!

**화이팅! 💪**